# AWS Machine Learning 목적별 가속기 튜토리얼
## [AWS Trainium](https://aws.amazon.com/machine-learning/trainium/)과 [AWS Inferentia](https://aws.amazon.com/machine-learning/inferentia/)를 [Amazon SageMaker](https://aws.amazon.com/sagemaker/)와 함께 사용하여 ML 워크로드를 최적화하는 방법 학습
## 1/3부 - 텍스트 분류를 위한 스팸/비스팸 데이터셋 준비

**SageMaker studio 커널: PyTorch 1.13 Python 3.9 CPU - ml.t3.medium** 

이 실습은 다음을 보여주는 엔드투엔드 튜토리얼의 일부입니다:
  - 1) 스팸/비스팸 데이터셋으로 텍스트 분류를 위한 데이터셋을 준비하는 방법
  - 2) Trn1 인스턴스를 사용하여 텍스트 분류(이진: 0=비스팸 1=스팸)를 위한 Bert Base 모델 미세 조정
  - 3) 학습된 모델을 컴파일하고 Inf2 인스턴스에 배포

## 1) 필요한 패키지 설치

In [ ]:
%pip install -r requirements.txt

## 2) 원시 데이터셋 다운로드 및 샘플 시각화

In [ ]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer

max_sen_len=256
model_id='bert-base-uncased'
dataset_path=os.path.join('datasets', 'spam')

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = 0

In [ ]:
dataset = load_dataset('Deysi/spam-detection-dataset')
dataset['test'].to_pandas().head()

## 3) 데이터셋을 패딩된 토큰으로 변환하고 샘플 시각화

In [ ]:
import torch
import numpy as np

labels = {'not_spam': 0, 'spam': 1}
def preprocess_function(examples):
    inp = tokenizer(examples["text"], padding='max_length', truncation=True, max_length=max_sen_len)
    inp['labels'] = [labels[e] for e in examples['label']]
    return inp

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset['train'].column_names,
)
tokenized_dataset = tokenized_dataset.with_format("torch")

## save dataset to disk
tokenized_dataset['train'].save_to_disk(os.path.join(dataset_path,"train"))
tokenized_dataset['test'].save_to_disk(os.path.join(dataset_path,"eval"))

In [ ]:
tokenized_dataset['train'].to_pandas().head()

## 4) 데이터셋을 S3에 업로드

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
print(f"Bucket: {bucket}")
sagemaker_session.upload_data(dataset_path, bucket=bucket, key_prefix=dataset_path)

## 5) (선택 사항) Collator가 모델을 위한 배치를 어떻게 생성하는지 확인

In [ ]:
import torch
from datasets import load_from_disk
from transformers import DefaultDataCollator

dataset = load_from_disk('datasets/spam/eval')
collator = DefaultDataCollator(return_tensors="pt")
it = iter(dataset)
batch = [next(it) for i in range(5)]
batch = collator(batch)
print("\n".join([f"{k}:\t{v.shape}" for k,v in batch.items()]))

## 6) 이제 모델을 미세 조정할 시간입니다

[학습 노트북 열기](02_ModelFineTuning.ipynb)